Modified from https://github.com/melanieabrams-pub/RH-seq_with_barcoding/blob/main/annotate_poolfile_for_barseq_v1.2_JSkerker.ipynb and 
https://github.com/melanieabrams-pub/kluyv_RH-seq_with_barcoding/blob/main/annotate_poolfile_kluyv.py

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

#makes viewing pandas tables better
pd.set_option('display.max_colwidth', 0)

### Load GFF annotations

In [5]:
#you don't need to change the path below - this is the most recent annotation file (5/12/22)
gff_file = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/Thermophilus_genes_and_promoters_12_19_2024_finalized_1.gff'

In [6]:
import pandas as pd

def count_unique_genes_gff(gff_file):
    # Read the GFF file
    gff_data = pd.read_csv(gff_file, sep='\t', comment='#')
    
    # Check if 'ID' column is present
    if 'ID' in gff_data.columns:
        # Extract the unique gene IDs from the 'ID' column
        unique_genes = gff_data['ID'].unique()
        
        # Filter out any NaN values
        unique_genes = unique_genes[~pd.isnull(unique_genes)]
        
        return len(unique_genes)
    else:
        return 0

gff_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/Thermophilus_genes_and_promoters_12_19_2024_finalized_1.gff'
total_genes = count_unique_genes_gff(gff_file_path)
print(f'Total number of genes in GFF file: {total_genes}')


Total number of genes in GFF file: 18220


In [7]:
gff = pd.read_csv(gff_file, sep='\t')

In [8]:
t = gff['scaffold'].unique()
print(t)

['chr1' 'chr2' 'chr3' 'chr4' 'chr5' 'chr6' 'chr7']


In [9]:
gff.head()

,scaffold,source,type,start,stop,strand,info,ID
0,chr1,JGI,promoter,381.0,1380.0,+,ID=gene_1,proteinId=2114025_promoter
1,chr1,JGI,CDS,1381.0,2142.0,+,ID=gene_1,proteinId=2114025
2,chr1,JGI,CDS,2744.0,3343.0,-,ID=gene_2,proteinId=2293935
3,chr1,JGI,CDS,3344.0,3817.0,-,ID=gene_3,proteinId=2293936
4,chr1,JGI,promoter,3344.0,4343.0,-,ID=gene_2,proteinId=2293935_promoter


In [10]:
gff

,scaffold,source,type,start,stop,strand,info,ID
0,chr1,JGI,promoter,381.0,1380.0,+,ID=gene_1,proteinId=2114025_promoter
1,chr1,JGI,CDS,1381.0,2142.0,+,ID=gene_1,proteinId=2114025
2,chr1,JGI,CDS,2744.0,3343.0,-,ID=gene_2,proteinId=2293935
3,chr1,JGI,CDS,3344.0,3817.0,-,ID=gene_3,proteinId=2293936
4,chr1,JGI,promoter,3344.0,4343.0,-,ID=gene_2,proteinId=2293935_promoter
...,...,...,...,...,...,...,...,...
18215,chr7,JGI,promoter,4098908.0,4099907.0,-,ID=gene_10838,proteinId=2114022_promoter
18216,chr7,JGI,CDS,4100606.0,4101293.0,-,ID=gene_10840,proteinId=2114023
18217,chr7,JGI,promoter,4101294.0,4102293.0,-,ID=gene_10840,proteinId=2114023_promoter
18218,chr7,JGI,CDS,4101637.0,4103479.0,-,ID=gene_10841,proteinId=2313233


### Match barcode insert locations to gene annotations

In [11]:
!pwd

/auto/sahara/namib/home/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants


In [12]:
#CHANGE IF NEEDED: load COMBINED poolfile generated in TNseq mapping step

poolfile = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile'

#CHANGE IF NEEDED: annotated pool file path to output

annotated_poolfile_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile_with_promoter_reannotated.csv'

In [13]:
df = pd.read_csv(poolfile,sep='\t')
df.head()

,barcode,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,All insert mappings
0,AGGCTCAAAGTACCTGTTAA,TTAACAGGTACTTTGAGCCT,33,33,chr4,+,1861183,Single,28,2,"[('chr4:1861183:+', 28), ('chr5:3550691:+', 1), ('chr7:1456504:+', 1), ('chr5:3256020:-', 1)]","{'Null:0:+': 0, 'insert:7:+': 1, 'plasmid:17:+': 1}"
1,TAGCCTAATTCTCTTTTAAT,ATTAAAAGAGAATTAGGCTA,26,26,chr1,+,6115762,Single,24,1,"[('chr1:6115762:+', 24), ('chr1:6914244:-', 1)]","{'Null:0:+': 0, 'plasmid:5371:+': 1}"
2,ATTACACGGGTCCATGATGA,TCATCATGGACCCGTGTAAT,1896,1896,chr2,+,4661375,Single,1746,49,"[('chr2:4661375:+', 1746), ('chr7:2406660:-', 8), ('chr2:4661376:+', 5), ('chr2:2256078:+', 4), ('chr2:4661374:+', 3), ('chr2:4327814:-', 2), ('chr1:987112:-', 2), ('chr3:2416675:+', 2), ('chr1:7974718:+', 2), ('chr2:4661377:+', 2), ('chr1:8833132:-', 2), ('chr2:491125:+', 2), ('chr2:2025832:+', 2), ('chr3:1548794:-', 2), ('chr7:2934458:-', 2), ('chr2:470383:-', 1), ('chr2:1789278:+', 1), ('chr1:3866952:+', 1), ('chr1:3821780:-', 1), ('chr7:221711:+', 1), ('chr2:2194574:-', 1), ('chr3:2413694:+', 1), ('chr3:740765:-', 1), ('chr7:1196515:-', 1), ('chr4:3414371:-', 1), ('chr6:72928:-', 1), ('chr1:6511162:-', 1), ('chr2:4314401:+', 1), ('chr1:2266535:-', 1), ('chr4:425807:+', 1), ('chr2:2592533:+', 1), ('chr1:9004567:-', 1), ('chr1:8069094:+', 1), ('chr5:2084104:-', 1), ('chr7:3471884:-', 1), ('chr7:602092:+', 1), ('chr1:9216225:+', 1), ('chr1:170029:-', 1), ('chr4:3545212:+', 1), ('chr6:1152798:+', 1), ('chr1:954628:+', 1), ('chr3:2887674:-', 1), ('chr2:5356625:-', 1), ('chr5:3565242:+', 1), ('chr1:4757905:+', 1), ('chr5:239032:-', 1), ('chr2:4994558:+', 1), ('chr2:3759614:+', 1), ('chr3:1300984:+', 1), ('chr1:5522532:+', 1), ('chr3:3815747:-', 1), ('chr1:7436906:+', 1), ('chr3:4887708:-', 1), ('chr2:1855143:+', 1), ('chr2:2305298:+', 1), ('chr2:4663687:-', 1), ('chr3:1633280:+', 1), ('chr7:2626447:-', 1), ('chr3:1265683:-', 1), ('chr2:2815637:+', 1), ('chr6:696895:-', 1), ('chr5:3325007:-', 1), ('chr1:3432198:+', 1), ('chr6:735571:-', 1), ('chr2:4085893:+', 1), ('chr1:3625441:+', 1), ('chr7:1943029:-', 1), ('chr7:1383965:+', 1), ('chr1:1333559:+', 1), ('chr1:6914244:-', 1), ('chr2:4022743:-', 1), ('chr3:1386264:-', 1), ('chr1:526973:-', 1), ('chr1:10013646:+', 1), ('chr3:3564347:+', 1), ('chr6:1351101:+', 1)]","{'Null:0:+': 0, 'insert:986:+': 1, 'insert:1877:+': 9, 'insert:13:+': 2, 'insert:890:+': 2, 'plasmid:4612:+': 1, 'insert:1658:-': 1, 'insert:510:+': 2, 'plasmid:5572:-': 1, 'plasmid:5596:-': 2, 'insert:10:+': 1, 'insert:1:+': 10, 'insert:7:+': 1, 'insert:478:+': 1, 'insert:14:+': 1, 'insert:26:+': 1, 'plasmid:5517:-': 1, 'insert:1953:+': 1, 'insert:4:+': 1, 'insert:939:+': 1, 'insert:1615:+': 1, 'insert:1580:+': 1, 'insert:494:-': 1, 'insert:9:+': 1, 'plasmid:1154:-': 1, 'insert:38:+': 1, 'insert:5:+': 1, 'insert:1748:-': 1, 'plasmid:302:-': 1}"
3,TGAGTTTTTGCGGGCCGACA,TGTCGGCCCGCAAAAACTCA,32,32,chr7,-,2325983,Single,29,1,"[('chr7:2325983:-', 29), ('chr1:237143:+', 1), ('chr3:3815747:-', 1)]","{'Null:0:+': 0, 'insert:12:+': 1}"
4,ACTGAACATCTTTGAACTCG,CGAGTTCAAAGATGTTCAGT,113,113,chr3,+,4987853,Concat,54,54,"[('chr3:4987853:+', 54), ('chr2:4661375:+', 1), ('chr4:1130987:+', 1), ('chr1:10114071:+', 1), ('chr1:4452506:+', 1), ('chr4:2704863:+', 1)]","{'Null:0:+': 0, 'insert:8:+': 48, 'insert:108:+': 1, 'insert:1:+': 2, 'insert:12:+': 1, 'insert:1454:-': 1, 'insert:5:+': 1}"


Check that each barcode in the dataframe is unique (the two numbers below should be the same):

In [14]:
print('Number of insertions in poolfile: {}'.format(df.shape[0]))
print('Number of unique barcodes in poolfile: {}'.format(len(df['barcode'].unique())))

Number of insertions in poolfile: 17154
Number of unique barcodes in poolfile: 17154


In [15]:
#df = df.set_index('transposonLocation') #This is what was there and it is showing error. The transposonlocation was changed to barcode in the next line
df = df.set_index('barcode')

In [16]:
df.columns

Index(['rcbarcode', 'nTot', 'n', 'scaffold', 'strand', 'pos', 'type',
       'nMainLocation', 'nInsert', 'All genomic mappings',
       'All insert mappings'],
      dtype='object')

In [17]:
df.head()

,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,All insert mappings
barcode,,,,,,,,,,,
AGGCTCAAAGTACCTGTTAA,TTAACAGGTACTTTGAGCCT,33,33,chr4,+,1861183,Single,28,2,"[('chr4:1861183:+', 28), ('chr5:3550691:+', 1), ('chr7:1456504:+', 1), ('chr5:3256020:-', 1)]","{'Null:0:+': 0, 'insert:7:+': 1, 'plasmid:17:+': 1}"
TAGCCTAATTCTCTTTTAAT,ATTAAAAGAGAATTAGGCTA,26,26,chr1,+,6115762,Single,24,1,"[('chr1:6115762:+', 24), ('chr1:6914244:-', 1)]","{'Null:0:+': 0, 'plasmid:5371:+': 1}"
ATTACACGGGTCCATGATGA,TCATCATGGACCCGTGTAAT,1896,1896,chr2,+,4661375,Single,1746,49,"[('chr2:4661375:+', 1746), ('chr7:2406660:-', 8), ('chr2:4661376:+', 5), ('chr2:2256078:+', 4), ('chr2:4661374:+', 3), ('chr2:4327814:-', 2), ('chr1:987112:-', 2), ('chr3:2416675:+', 2), ('chr1:7974718:+', 2), ('chr2:4661377:+', 2), ('chr1:8833132:-', 2), ('chr2:491125:+', 2), ('chr2:2025832:+', 2), ('chr3:1548794:-', 2), ('chr7:2934458:-', 2), ('chr2:470383:-', 1), ('chr2:1789278:+', 1), ('chr1:3866952:+', 1), ('chr1:3821780:-', 1), ('chr7:221711:+', 1), ('chr2:2194574:-', 1), ('chr3:2413694:+', 1), ('chr3:740765:-', 1), ('chr7:1196515:-', 1), ('chr4:3414371:-', 1), ('chr6:72928:-', 1), ('chr1:6511162:-', 1), ('chr2:4314401:+', 1), ('chr1:2266535:-', 1), ('chr4:425807:+', 1), ('chr2:2592533:+', 1), ('chr1:9004567:-', 1), ('chr1:8069094:+', 1), ('chr5:2084104:-', 1), ('chr7:3471884:-', 1), ('chr7:602092:+', 1), ('chr1:9216225:+', 1), ('chr1:170029:-', 1), ('chr4:3545212:+', 1), ('chr6:1152798:+', 1), ('chr1:954628:+', 1), ('chr3:2887674:-', 1), ('chr2:5356625:-', 1), ('chr5:3565242:+', 1), ('chr1:4757905:+', 1), ('chr5:239032:-', 1), ('chr2:4994558:+', 1), ('chr2:3759614:+', 1), ('chr3:1300984:+', 1), ('chr1:5522532:+', 1), ('chr3:3815747:-', 1), ('chr1:7436906:+', 1), ('chr3:4887708:-', 1), ('chr2:1855143:+', 1), ('chr2:2305298:+', 1), ('chr2:4663687:-', 1), ('chr3:1633280:+', 1), ('chr7:2626447:-', 1), ('chr3:1265683:-', 1), ('chr2:2815637:+', 1), ('chr6:696895:-', 1), ('chr5:3325007:-', 1), ('chr1:3432198:+', 1), ('chr6:735571:-', 1), ('chr2:4085893:+', 1), ('chr1:3625441:+', 1), ('chr7:1943029:-', 1), ('chr7:1383965:+', 1), ('chr1:1333559:+', 1), ('chr1:6914244:-', 1), ('chr2:4022743:-', 1), ('chr3:1386264:-', 1), ('chr1:526973:-', 1), ('chr1:10013646:+', 1), ('chr3:3564347:+', 1), ('chr6:1351101:+', 1)]","{'Null:0:+': 0, 'insert:986:+': 1, 'insert:1877:+': 9, 'insert:13:+': 2, 'insert:890:+': 2, 'plasmid:4612:+': 1, 'insert:1658:-': 1, 'insert:510:+': 2, 'plasmid:5572:-': 1, 'plasmid:5596:-': 2, 'insert:10:+': 1, 'insert:1:+': 10, 'insert:7:+': 1, 'insert:478:+': 1, 'insert:14:+': 1, 'insert:26:+': 1, 'plasmid:5517:-': 1, 'insert:1953:+': 1, 'insert:4:+': 1, 'insert:939:+': 1, 'insert:1615:+': 1, 'insert:1580:+': 1, 'insert:494:-': 1, 'insert:9:+': 1, 'plasmid:1154:-': 1, 'insert:38:+': 1, 'insert:5:+': 1, 'insert:1748:-': 1, 'plasmid:302:-': 1}"
TGAGTTTTTGCGGGCCGACA,TGTCGGCCCGCAAAAACTCA,32,32,chr7,-,2325983,Single,29,1,"[('chr7:2325983:-', 29), ('chr1:237143:+', 1), ('chr3:3815747:-', 1)]","{'Null:0:+': 0, 'insert:12:+': 1}"
ACTGAACATCTTTGAACTCG,CGAGTTCAAAGATGTTCAGT,113,113,chr3,+,4987853,Concat,54,54,"[('chr3:4987853:+', 54), ('chr2:4661375:+', 1), ('chr4:1130987:+', 1), ('chr1:10114071:+', 1), ('chr1:4452506:+', 1), ('chr4:2704863:+', 1)]","{'Null:0:+': 0, 'insert:8:+': 48, 'insert:108:+': 1, 'insert:1:+': 2, 'insert:12:+': 1, 'insert:1454:-': 1, 'insert:5:+': 1}"


In [18]:
df=df.drop(df.columns[-1],axis=1)

In [19]:
# df.columns = ['rcbarcode','nTot', 'n', 'scaffold', 'strand', 'pos', 'type',
#        'nMainLocation', 'nInsert', 'All genomic mappings',
#        'All insert mappings']

In [20]:
df.head()

,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings
barcode,,,,,,,,,,
AGGCTCAAAGTACCTGTTAA,TTAACAGGTACTTTGAGCCT,33,33,chr4,+,1861183,Single,28,2,"[('chr4:1861183:+', 28), ('chr5:3550691:+', 1), ('chr7:1456504:+', 1), ('chr5:3256020:-', 1)]"
TAGCCTAATTCTCTTTTAAT,ATTAAAAGAGAATTAGGCTA,26,26,chr1,+,6115762,Single,24,1,"[('chr1:6115762:+', 24), ('chr1:6914244:-', 1)]"
ATTACACGGGTCCATGATGA,TCATCATGGACCCGTGTAAT,1896,1896,chr2,+,4661375,Single,1746,49,"[('chr2:4661375:+', 1746), ('chr7:2406660:-', 8), ('chr2:4661376:+', 5), ('chr2:2256078:+', 4), ('chr2:4661374:+', 3), ('chr2:4327814:-', 2), ('chr1:987112:-', 2), ('chr3:2416675:+', 2), ('chr1:7974718:+', 2), ('chr2:4661377:+', 2), ('chr1:8833132:-', 2), ('chr2:491125:+', 2), ('chr2:2025832:+', 2), ('chr3:1548794:-', 2), ('chr7:2934458:-', 2), ('chr2:470383:-', 1), ('chr2:1789278:+', 1), ('chr1:3866952:+', 1), ('chr1:3821780:-', 1), ('chr7:221711:+', 1), ('chr2:2194574:-', 1), ('chr3:2413694:+', 1), ('chr3:740765:-', 1), ('chr7:1196515:-', 1), ('chr4:3414371:-', 1), ('chr6:72928:-', 1), ('chr1:6511162:-', 1), ('chr2:4314401:+', 1), ('chr1:2266535:-', 1), ('chr4:425807:+', 1), ('chr2:2592533:+', 1), ('chr1:9004567:-', 1), ('chr1:8069094:+', 1), ('chr5:2084104:-', 1), ('chr7:3471884:-', 1), ('chr7:602092:+', 1), ('chr1:9216225:+', 1), ('chr1:170029:-', 1), ('chr4:3545212:+', 1), ('chr6:1152798:+', 1), ('chr1:954628:+', 1), ('chr3:2887674:-', 1), ('chr2:5356625:-', 1), ('chr5:3565242:+', 1), ('chr1:4757905:+', 1), ('chr5:239032:-', 1), ('chr2:4994558:+', 1), ('chr2:3759614:+', 1), ('chr3:1300984:+', 1), ('chr1:5522532:+', 1), ('chr3:3815747:-', 1), ('chr1:7436906:+', 1), ('chr3:4887708:-', 1), ('chr2:1855143:+', 1), ('chr2:2305298:+', 1), ('chr2:4663687:-', 1), ('chr3:1633280:+', 1), ('chr7:2626447:-', 1), ('chr3:1265683:-', 1), ('chr2:2815637:+', 1), ('chr6:696895:-', 1), ('chr5:3325007:-', 1), ('chr1:3432198:+', 1), ('chr6:735571:-', 1), ('chr2:4085893:+', 1), ('chr1:3625441:+', 1), ('chr7:1943029:-', 1), ('chr7:1383965:+', 1), ('chr1:1333559:+', 1), ('chr1:6914244:-', 1), ('chr2:4022743:-', 1), ('chr3:1386264:-', 1), ('chr1:526973:-', 1), ('chr1:10013646:+', 1), ('chr3:3564347:+', 1), ('chr6:1351101:+', 1)]"
TGAGTTTTTGCGGGCCGACA,TGTCGGCCCGCAAAAACTCA,32,32,chr7,-,2325983,Single,29,1,"[('chr7:2325983:-', 29), ('chr1:237143:+', 1), ('chr3:3815747:-', 1)]"
ACTGAACATCTTTGAACTCG,CGAGTTCAAAGATGTTCAGT,113,113,chr3,+,4987853,Concat,54,54,"[('chr3:4987853:+', 54), ('chr2:4661375:+', 1), ('chr4:1130987:+', 1), ('chr1:10114071:+', 1), ('chr1:4452506:+', 1), ('chr4:2704863:+', 1)]"


In [21]:
#this takes a few minutes
def get_gene_by_bc_position(pos):
    
    gene_list = [scaffold_gff.loc[interval]['ID'] for interval in idx if pos in interval]
    
    #barcode position not in any genes
    if not gene_list:
        return ''
    #barcode position is in one gene
    if len(gene_list) == 1:
        return gene_list[0]
    #barcode position is in multiple genes (should be rare)
    else:
        return ';'.join(gene_list)

barcode_to_gene_dict = {}

for scaffold in sorted(gff['scaffold'].unique()):
    
    scaffold_gff = gff[gff['scaffold']==scaffold]
    scaffold_barcodes = df[df['scaffold']==scaffold]
    
    #divide scaffold_gff.shape by 2 here to get number of genes (original shape includes promoters)
    print('Mapping {} barcodes on {} to {} genes on {}'.format(scaffold_barcodes.shape[0], scaffold,
                                                              round(scaffold_gff.shape[0]/2), scaffold))

    #turn the start and stop of each gene into an interval
    v = scaffold_gff.loc[:, 'start':'stop'].apply(tuple, 1).tolist()
    idx = pd.IntervalIndex.from_tuples(v, closed='both') 
    scaffold_gff = scaffold_gff.set_index(idx)
    
    #for each barcode, check if it's in each start/stop interval
    barcode_to_gene_dict.update(scaffold_barcodes['pos'].apply(get_gene_by_bc_position).to_dict())
    
df['gene'] = df.index.map(barcode_to_gene_dict)

Mapping 5423 barcodes on chr1 to 2712 genes on chr1


TypeError: sequence item 0: expected str instance, Series found

In [22]:
def get_gene_by_bc_position(pos):
    gene_list = [str(scaffold_gff.loc[interval]['ID']) for interval in idx if pos in interval]
    
    # Barcode position not in any genes
    if not gene_list:
        return ''
    # Barcode position is in one gene
    if len(gene_list) == 1:
        return gene_list[0]
    # Barcode position is in multiple genes (should be rare)
    else:
        return ';'.join(gene_list)

barcode_to_gene_dict = {}

for scaffold in sorted(gff['scaffold'].unique()):
    scaffold_gff = gff[gff['scaffold'] == scaffold]
    scaffold_barcodes = df[df['scaffold'] == scaffold]
    
    # Divide scaffold_gff.shape by 2 here to get number of genes
    print('Mapping {} barcodes on {} to {} genes on {}'.format(scaffold_barcodes.shape[0], scaffold,
                                                              round(scaffold_gff.shape[0] / 2), scaffold))
    
    # Turn the start and stop of each gene into an interval
    v = scaffold_gff.loc[:, 'start':'stop'].apply(tuple, 1).tolist()
    idx = pd.IntervalIndex.from_tuples(v, closed='both') 
    scaffold_gff = scaffold_gff.set_index(idx)
    
    # For each barcode, check if it's in each start/stop interval
    barcode_to_gene_dict.update(scaffold_barcodes['pos'].apply(get_gene_by_bc_position).to_dict())
    
df['gene'] = df.index.map(barcode_to_gene_dict)


Mapping 5423 barcodes on chr1 to 2712 genes on chr1
Mapping 2949 barcodes on chr2 to 1430 genes on chr2
Mapping 2287 barcodes on chr3 to 1210 genes on chr3
Mapping 1664 barcodes on chr4 to 1070 genes on chr4
Mapping 1302 barcodes on chr5 to 985 genes on chr5
Mapping 1242 barcodes on chr6 to 793 genes on chr6
Mapping 2287 barcodes on chr7 to 910 genes on chr7


In [19]:
import pandas as pd

def get_gene_by_bc_position(pos):
    gene_list = [scaffold_gff.loc[interval]['ID'] for interval in idx if pos in interval]
    
    #barcode position not in any genes
    if not gene_list:
        return ''
    #barcode position is in one gene
    if len(gene_list) == 1:
        return gene_list[0]
    #barcode position is in multiple genes (should be rare)
    else:
        return ';'.join(gene_list)

barcode_to_gene_dict = {}

for scaffold in sorted(gff['scaffold'].unique()):
    
    scaffold_gff = gff[gff['scaffold'] == scaffold]
    scaffold_barcodes = df[df['scaffold'] == scaffold]
    
    # Count unique genes
    unique_genes_count = scaffold_gff['ID'].nunique()
    
    print(f'Mapping {scaffold_barcodes.shape[0]} barcodes on {scaffold} to {unique_genes_count} genes on {scaffold}')

    # Turn the start and stop of each gene into an interval
    v = scaffold_gff.loc[:, 'start':'stop'].apply(tuple, axis=1).tolist()
    idx = pd.IntervalIndex.from_tuples(v, closed='both') 
    scaffold_gff = scaffold_gff.set_index(idx)
    
    # For each barcode, check if it's in each start/stop interval
    barcode_to_gene_dict.update(scaffold_barcodes['pos'].apply(get_gene_by_bc_position).to_dict())
    
df['gene'] = df.index.map(barcode_to_gene_dict)


Mapping 5423 barcodes on chr1 to 2712 genes on chr1
Mapping 2949 barcodes on chr2 to 1430 genes on chr2
Mapping 2287 barcodes on chr3 to 1210 genes on chr3
Mapping 1664 barcodes on chr4 to 1070 genes on chr4
Mapping 1302 barcodes on chr5 to 985 genes on chr5
Mapping 1242 barcodes on chr6 to 793 genes on chr6
Mapping 2287 barcodes on chr7 to 910 genes on chr7


In [23]:
#check that a 'gene' column has been added to df

df.head()

,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,gene
barcode,,,,,,,,,,,
AGGCTCAAAGTACCTGTTAA,TTAACAGGTACTTTGAGCCT,33,33,chr4,+,1861183,Single,28,2,"[('chr4:1861183:+', 28), ('chr5:3550691:+', 1), ('chr7:1456504:+', 1), ('chr5:3256020:-', 1)]",proteinId=2093740_promoter;proteinId=2306381
TAGCCTAATTCTCTTTTAAT,ATTAAAAGAGAATTAGGCTA,26,26,chr1,+,6115762,Single,24,1,"[('chr1:6115762:+', 24), ('chr1:6914244:-', 1)]",proteinId=2297294
ATTACACGGGTCCATGATGA,TCATCATGGACCCGTGTAAT,1896,1896,chr2,+,4661375,Single,1746,49,"[('chr2:4661375:+', 1746), ('chr7:2406660:-', 8), ('chr2:4661376:+', 5), ('chr2:2256078:+', 4), ('chr2:4661374:+', 3), ('chr2:4327814:-', 2), ('chr1:987112:-', 2), ('chr3:2416675:+', 2), ('chr1:7974718:+', 2), ('chr2:4661377:+', 2), ('chr1:8833132:-', 2), ('chr2:491125:+', 2), ('chr2:2025832:+', 2), ('chr3:1548794:-', 2), ('chr7:2934458:-', 2), ('chr2:470383:-', 1), ('chr2:1789278:+', 1), ('chr1:3866952:+', 1), ('chr1:3821780:-', 1), ('chr7:221711:+', 1), ('chr2:2194574:-', 1), ('chr3:2413694:+', 1), ('chr3:740765:-', 1), ('chr7:1196515:-', 1), ('chr4:3414371:-', 1), ('chr6:72928:-', 1), ('chr1:6511162:-', 1), ('chr2:4314401:+', 1), ('chr1:2266535:-', 1), ('chr4:425807:+', 1), ('chr2:2592533:+', 1), ('chr1:9004567:-', 1), ('chr1:8069094:+', 1), ('chr5:2084104:-', 1), ('chr7:3471884:-', 1), ('chr7:602092:+', 1), ('chr1:9216225:+', 1), ('chr1:170029:-', 1), ('chr4:3545212:+', 1), ('chr6:1152798:+', 1), ('chr1:954628:+', 1), ('chr3:2887674:-', 1), ('chr2:5356625:-', 1), ('chr5:3565242:+', 1), ('chr1:4757905:+', 1), ('chr5:239032:-', 1), ('chr2:4994558:+', 1), ('chr2:3759614:+', 1), ('chr3:1300984:+', 1), ('chr1:5522532:+', 1), ('chr3:3815747:-', 1), ('chr1:7436906:+', 1), ('chr3:4887708:-', 1), ('chr2:1855143:+', 1), ('chr2:2305298:+', 1), ('chr2:4663687:-', 1), ('chr3:1633280:+', 1), ('chr7:2626447:-', 1), ('chr3:1265683:-', 1), ('chr2:2815637:+', 1), ('chr6:696895:-', 1), ('chr5:3325007:-', 1), ('chr1:3432198:+', 1), ('chr6:735571:-', 1), ('chr2:4085893:+', 1), ('chr1:3625441:+', 1), ('chr7:1943029:-', 1), ('chr7:1383965:+', 1), ('chr1:1333559:+', 1), ('chr1:6914244:-', 1), ('chr2:4022743:-', 1), ('chr3:1386264:-', 1), ('chr1:526973:-', 1), ('chr1:10013646:+', 1), ('chr3:3564347:+', 1), ('chr6:1351101:+', 1)]",proteinId=2302542
TGAGTTTTTGCGGGCCGACA,TGTCGGCCCGCAAAAACTCA,32,32,chr7,-,2325983,Single,29,1,"[('chr7:2325983:-', 29), ('chr1:237143:+', 1), ('chr3:3815747:-', 1)]",
ACTGAACATCTTTGAACTCG,CGAGTTCAAAGATGTTCAGT,113,113,chr3,+,4987853,Concat,54,54,"[('chr3:4987853:+', 54), ('chr2:4661375:+', 1), ('chr4:1130987:+', 1), ('chr1:10114071:+', 1), ('chr1:4452506:+', 1), ('chr4:2704863:+', 1)]",proteinId=2305679;proteinId=2305680_promoter


### This line is not necessary in the running. Continue to the next line to save the dataframe

In [ ]:
print('{} barcodes not in coding sequences'.format(df[df['gene']==''].shape[0]))
print()

print('{} barcodes in coding sequences'.format(df[df['gene']!=''].shape[0]))
print('{} barcodes in Kl coding sequences'.format(df[df['gene'].str.startswith('kl',na=False)].shape[0]))
print('{} barcodes in Km coding sequences'.format(df[df['gene'].str.startswith('km',na=False)].shape[0]))

In [24]:
df.to_csv(annotated_poolfile_path, sep='\t')

In [ ]:
gff[gff['ID']=='kmYHR034C']